In [ ]:
!pip install sagemaker-experiments

In [2]:
import sagemaker
import boto3
import numpy as np                                
import pandas as pd                               
import os                                         
from sagemaker import get_execution_role
from datetime import datetime

# Get default bucket
bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/DEMO-xgboost-dm'

# Get SageMaker Execution Role
role = get_execution_role()
region = boto3.Session().region_name

# SageMaker Session
sess = sagemaker.session.Session()

---

## Training

To train a model in SageMaker, you create a training job. The training job includes the following information:

* The Amazon Elastic Container Registry path where the training code is stored.

* The URL of the Amazon Simple Storage Service (Amazon S3) bucket where you've stored the training data.

* The compute resources that you want SageMaker to use for model training. Compute resources are ML compute instances that are managed by SageMaker.

* The URL of the S3 bucket where you want to store the output of the job.

SageMaker built-in algorithms require the least effort and scale if the data set is large and significant resources are needed to train and deploy the model. For this use case, we will use the built-in xgboost algorithm in SageMaker.

`xgboost` is an extremely popular, open-source package for gradient boosted trees.  It is computationally powerful, fully featured, and has been successfully used in many machine learning competitions.  Let's start with a simple `xgboost` model, trained using Amazon SageMaker's managed, distributed training framework.

In [3]:
container = sagemaker.image_uris.retrieve(region=region, framework='xgboost', version='latest')

In [4]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')
s3_input_test ='s3://{}/{}/test/test.csv'.format(bucket, prefix)

## Create an Experiment

To ensure we are able to keep track of our parameters and metrics that correspond to the training job, we create an Experiment and add this Training job to a Trial within that Experiment. 

Experiments are organized as -
```
Experiment
    Trial
        Trial Component 1
        Trial Component 2
        ...
```     
In this notebook, each time we run the Training job, it will correspond to a Trial Component and we organize that into Trials that represent each iterative experiment we run. 

In [5]:
current_time = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")

### Create the Experiment

In [6]:
from smexperiments.experiment import Experiment

sm = boto3.client('sagemaker')
xgboost_experiment = Experiment.create(experiment_name=f'xgboost-banking-dataset-experiment-{current_time}')

### Create the Trial

In [7]:
trial = xgboost_experiment.create_trial(trial_name=f'trial-{current_time}')

An estimator is a high level interface for SageMaker training. We will create an estimator object by supplying the required parameters, such as IAM role, compute instance count and type. and the S3 output path. 

We also supply hyperparameters for the algoirthm and then call its fit() method to start training the model.

In [8]:
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.m4.xlarge',
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    sagemaker_session=sess
)

xgb.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    silent=0,
    objective='binary:logistic',
    num_round=100
)

xgb.fit(
    inputs = {
        'train': s3_input_train, 
        'validation': s3_input_validation
    },
    experiment_config = {
        "ExperimentName": xgboost_experiment.experiment_name,
        "TrialName": trial.trial_name,
        "TrialComponentDisplayName": "XGB-Training"
    }
) 

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: xgboost-2022-09-27-20-40-33-834


ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: No S3 objects found under S3 URL "s3://sagemaker-us-east-2-952750099313/sagemaker/DEMO-xgboost-dm/train" given in input data source. Please ensure that the bucket exists in the selected region (us-east-2), that objects exist under that S3 prefix, and that the role "arn:aws:iam::952750099313:role/service-role/AmazonSageMaker-ExecutionRole-20210518T140197" has "s3:ListBucket" permissions on bucket "sagemaker-us-east-2-952750099313".

## Register model

## Deploy model